<a href="https://colab.research.google.com/github/AnanyaUp/Quantum-Machine-Learning/blob/main/QML_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install qiskit
!pip install qiskit-machine-learning
!pip install pennylane pennylane[torch]    # PyTorch support
!pip install pennylane[tensorflow]        # TensorFlow support

import pennylane as qml
print(qml.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 58.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3
  Attempting uninstall: qiskit
    Found existing installation: qiskit 2.2.3
    Uninstalling qiskit-2.2.3:
      Successfully uninstalled qiskit-2.2.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 1.8 MB/s et

/usr/local/lib/python3.12/dist-packages/pennylane/__init__.py:209: RuntimeWarning: PennyLane is not yet compatible with JAX versions > 0.6.2. You have version 0.7.2 installed. Please downgrade JAX to 0.6.2 to avoid runtime errors using python -m pip install jax~=0.6.0 jaxlib~=0.6.0
  warnings.warn(


In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import pandas
import pandas as pd

# Step 3: Set the path to your CSV files
base_path = '/content/drive/MyDrive/Titanic '

# Step 4: Load the CSV files
train = pd.read_csv(base_path + '/train.csv')
test = pd.read_csv(base_path + '/test.csv')
gender_submission = pd.read_csv(base_path + '/gender_submission.csv')

# Step 5: Check the data
print("Train shape:", train.shape)
print("Test shape:", test.shape)
print("Gender Submission shape:", gender_submission.shape)

# Optional: Display first few rows
train.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train shape: (891, 12)
Test shape: (418, 11)
Gender Submission shape: (418, 2)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 20.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from math import asin, sqrt
from functools import reduce
import matplotlib.pyplot as plt

# Import latest Qiskit components
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.primitives import StatevectorSampler
from qiskit.quantum_info import Statevector

In [ ]:
# Helper function to convert probability to angle
def prob_to_angle(prob):
    """
    Converts a given P(psi) value into an equivalent theta value.
    """
    return 2 * asin(sqrt(prob))


In [ ]:
# Calculate total passengers and survivors
cnt_all = len(train) # total passengers
survivors = train[train.Survived.eq(1)] # passenger who survived
cnt_survivors = len(survivors) # probability survival


In [ ]:
# Calculate the prior probability
prob_survival = len(survivors) / cnt_all
print("The prior probability to survive is:", round(prob_survival, 2))

The prior probability to survive is: 0.38


In [ ]:
# Get modifier for passenger class
def get_modifier_pclass(pclass):
    """Calculate modifier based on passenger class"""
    # Number of passengers with the same pclass who survived
    cnt_surv_pclass = len(survivors[survivors.Pclass.eq(pclass)])

    # Backward probability P(Pclass|Survived)
    p_cl_surv = cnt_surv_pclass / cnt_survivors

    # Probability of the evidence P(Pclass)
    p_cl = len(train[train.Pclass.eq(pclass)]) / cnt_all

    return p_cl_surv / p_cl

In [ ]:
# Get modifier for passenger gender
def get_modifier_sex(sex):
    """Calculate modifier based on passenger gender"""
    # Number of passengers with the same gender who survived
    cnt_surv_sex = len(survivors[survivors.Sex.eq(sex)])

    # Backward probability P(Sex|Survived)
    p_sex_surv = cnt_surv_sex / cnt_survivors

    # Probability of the evidence P(Sex)
    p_sex = len(train[train.Sex.eq(sex)]) / cnt_all

    return p_sex_surv / p_sex

In [ ]:
# Pre-processing function
def pre_process(passenger):
    """
    passenger -- the Pandas dataframe row of the passenger
    returns a list of modifiers, like this [modifier_a, modifier_b, ...]
    """
    return [
        get_modifier_pclass(passenger["Pclass"]),
        get_modifier_sex(passenger["Sex"]),
    ]

In [ ]:
# Parameterized Quantum Circuit (PQC)
def pqc(backend, prior, modifiers, shots=1, hist=False, measure=False):
    """
    Main quantum circuit implementing Quantum Naive Bayes
    """
    # Prepare the circuit with qubits and classical register
    qr = QuantumRegister(7)
    # Target qubit + 2 auxiliary qubits + 4 trunk qubits for intermediate calculations
    cr = ClassicalRegister(1)
    qc = QuantumCircuit(qr, cr) if measure else QuantumCircuit(qr)

    # Define qubit positions
    '''
    Qubits 0-3: Trunk qubits for intermediate calculations

    Qubit 4: Auxiliary qubit holding half the prior probability

    Qubit 5: Auxiliary qubit holding full prior probability

    Qubit 6: Target qubit representing final posterior probability

    '''

    trunks = 3
    aux_half = trunks + 1  # Position 4
    aux_full = trunks + 2  # Position 5
    target = trunks + 3    # Position 6

    # Apply prior to the target qubit
    #  Encodes prior probability (0.38) into target qubit
    qc.ry(prob_to_angle(prior), target)

    # Work with the remainder

    qc.x(target)

    # Apply prior to full auxiliary qubit
    qc.cry(prob_to_angle(prior / (1 - prior)), target, aux_full)

    # Uses remainder to set aux_full qubit with same prior probability
    # Flips target to work with remainder (1 - prior = 0.62)

    # Work with the remainder
    qc.cx(aux_full, target)

    # Apply 0.5*prior to aux_half qubit
    qc.cry(prob_to_angle(0.5 * prior / (1 - (2 * prior))), target, aux_half)

    # Rearrange states to separated qubits
    qc.x(target)
    qc.cx(aux_full, target)

    '''
    For modifiers > 1.0 (increasing survival probability):

      Use controlled rotation to "add" probability from aux_full to target

      This increases the posterior probability beyond the prior

    '''
     # Sort modifiers (start with larger one)
    sorted_modifiers = sorted(modifiers)

    # Apply modifiers
    for step in range(0, len(modifiers)):
        if sorted_modifiers[step] > 1:
            # For modifier greater than 1.0
            qc.cry(prob_to_angle(min(1, sorted_modifiers[step] - 1)), aux_full, target)

            # Separate the aux_full and the target qubit
            # ccx = toffoli gate -> separate overlapping probability states to avoid double-counting.
            qc.ccx(target, aux_full, 0)
            qc.ccx(target, 0, aux_full)

            if step == 0:
                # Refill the aux_full qubit for potential second modifier
                qc.cry(prob_to_angle(min(1, (sorted_modifiers[step] - 1) * 2 * 2)),
                       aux_half, aux_full)
        else:

            '''
            For modifiers < 1.0 (decreasing survival probability):

              Remove probability from target qubit proportional to (1 - modifier)

              This decreases the posterior probability below the prior
            '''

            # For modifier less than 1.0
            # Apply modifier to the target qubit
            qc.cry(prob_to_angle(1 - sorted_modifiers[step]), target, step * 2)
            qc.cx(step * 2, target)

            if step == 0:
                # Apply modifier to full auxiliary qubit
                qc.cry(prob_to_angle(1 - sorted_modifiers[step]), aux_full, step * 2 + 1)

                # Unentangle the full auxiliary from trunk
                qc.cx(step * 2 + 1, aux_full)

    # Execute the circuit
    if measure:
        # Add measurement to the target qubit
        qc.measure(qr[target], cr[0])

        # Use StatevectorSampler for measurement
        sampler = StatevectorSampler()
        job = sampler.run([qc], shots=shots)
        result = job.result()

        # FIXED: Properly extract counts from SamplerResult
        # Access the first (and only) pub result
        pub_result = result[0]

        # Get the counts from the data
        counts_dict = {}
        if hasattr(pub_result.data, 'c'):
            # For classical register measurements
            measurements = pub_result.data.c.array
        else:
            # Alternative access method
            measurements = pub_result.data.meas.array

        # Convert measurements to counts
        for measurement in measurements:
            bit_string = str(int(measurement))
            counts_dict[bit_string] = counts_dict.get(bit_string, 0) + 1

        return plot_histogram(counts_dict, figsize=(12, 4)) if hist else counts_dict
    else:
        # For statevector simulation without measurement
        from qiskit.quantum_info import Statevector
        statevector = Statevector.from_instruction(qc)
        return statevector


In [ ]:
# Alternative method using AerSimulator for measurements
def pqc_aer(backend, prior, modifiers, shots=1, hist=False):
    """
    Alternative implementation using AerSimulator for reliable measurement results
    """
    # Prepare the circuit with qubits and classical register
    qr = QuantumRegister(7)
    cr = ClassicalRegister(1)
    qc = QuantumCircuit(qr, cr)

    # Define qubit positions
    trunks = 3
    aux_half = trunks + 1  # Position 4
    aux_full = trunks + 2  # Position 5
    target = trunks + 3    # Position 6

    # Apply prior to the target qubit
    qc.ry(prob_to_angle(prior), target)

    # Work with the remainder
    qc.x(target)

    # Apply prior to full auxiliary qubit
    qc.cry(prob_to_angle(prior / (1 - prior)), target, aux_full)

    # Work with the remainder
    qc.cx(aux_full, target)

    # Apply 0.5*prior to aux_half qubit
    qc.cry(prob_to_angle(0.5 * prior / (1 - (2 * prior))), target, aux_half)

    # Rearrange states to separated qubits
    qc.x(target)
    qc.cx(aux_full, target)

    # Sort modifiers (start with larger one)
    sorted_modifiers = sorted(modifiers)

    # Apply modifiers
    for step in range(0, len(modifiers)):
        if sorted_modifiers[step] > 1:
            # For modifier greater than 1.0
            qc.cry(prob_to_angle(min(1, sorted_modifiers[step] - 1)), aux_full, target)

            # Separate the aux_full and the target qubit
            qc.ccx(target, aux_full, 0)
            qc.ccx(target, 0, aux_full)

            if step == 0:
                # Refill the aux_full qubit for potential second modifier
                qc.cry(prob_to_angle(min(1, (sorted_modifiers[step] - 1) * 2 * 2)),
                       aux_half, aux_full)
        else:
            # For modifier less than 1.0
            # Apply modifier to the target qubit
            qc.cry(prob_to_angle(1 - sorted_modifiers[step]), target, step * 2)
            qc.cx(step * 2, target)

            if step == 0:
                # Apply modifier to full auxiliary qubit
                qc.cry(prob_to_angle(1 - sorted_modifiers[step]), aux_full, step * 2 + 1)

                # Unentangle the full auxiliary from trunk
                qc.cx(step * 2 + 1, aux_full)

    # Add measurement to the target qubit
    qc.measure(qr[target], cr[0])

    # Run on AerSimulator
    job = backend.run(qc, shots=shots) # Executes circuit on simulator
    result = job.result() # Returns measurement counts as dictionary
    counts = result.get_counts()

    return plot_histogram(counts, figsize=(12, 4)) if hist else counts



In [ ]:
# Post-processing function
# Converts quantum measurement results to binary classification:

def post_process(counts):
    """
    counts -- the result of the quantum circuit execution
    returns the prediction
    """
    # Handle both dictionary formats
    if isinstance(counts, dict) and counts:
        # Get the most frequent measurement
        max_count = max(counts.values())
        for bit_string, count in counts.items():
            if count == max_count:
                return int(bit_string)
    return 0  # Default prediction

In [ ]:
# Run function for batch processing
def run(f_classify, data):
    """Apply classifier to all data points"""
    return [f_classify(data.iloc[i]) for i in range(0, len(data))]


In [ ]:
# Classifier evaluation function
def classifier_report(name, run_func, classifier, data, labels):
    """Evaluate classifier performance"""
    predictions = run_func(classifier, data)

    # Calculate metrics
    tp = sum(1 for i, pred in enumerate(predictions) if pred == 1 and labels.iloc[i] == 1)
    tn = sum(1 for i, pred in enumerate(predictions) if pred == 0 and labels.iloc[i] == 0)
    fp = sum(1 for i, pred in enumerate(predictions) if pred == 1 and labels.iloc[i] == 0)
    fn = sum(1 for i, pred in enumerate(predictions) if pred == 0 and labels.iloc[i] == 1)

    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
    npv = tn / (tn + fn) if (tn + fn) > 0 else 0

    accuracy = (tp + tn) / len(predictions)

    print(f"The precision score of the {name} classifier is {precision:.2f}")
    print(f"The recall score of the {name} classifier is {recall:.2f}")
    print(f"The specificity score of the {name} classifier is {specificity:.2f}")
    print(f"The npv score of the {name} classifier is {npv:.2f}")
    print(f"The information level is: {accuracy:.2f}")

In [ ]:
# Create backend using AerSimulator
backend = AerSimulator()

In [ ]:
# Test with example passengers
print("\nTesting with example passengers:")
print("Female passenger with 1st class ticket:", pre_process(train.iloc[52]))
print("Male passenger with 3rd class ticket:", pre_process(train.iloc[26]))


Testing with example passengers:
Female passenger with 1st class ticket: [1.6403508771929822, 1.9332048273550118]
Male passenger with 3rd class ticket: [0.6314181584306999, 0.49215543190732464]


In [ ]:
# Test single prediction using AerSimulator method (more reliable)
test_passenger = train.iloc[52]
modifiers = pre_process(test_passenger)
result = pqc_aer(backend, prob_survival, modifiers, shots=1000, hist=False)
print(f"\nQuantum prediction result: {result}")
prediction = post_process(result)
print(f"Prediction for passenger: {prediction}")


Quantum prediction result: {'0': 56, '1': 944}
Prediction for passenger: 1


In [ ]:
# Run complete classifier evaluation using AerSimulator
print("\n=== Quantum Naive Bayes Classifier Evaluation ===")
classifier_report(
    "QuantumNaiveBayes",
    run,
    lambda passenger: post_process(pqc_aer(backend, prob_survival, pre_process(passenger),
                                          shots=100)),
    train.head(50),  # Use first 50 samples for faster testing
    train['Survived'].head(50)
)


=== Quantum Naive Bayes Classifier Evaluation ===
The precision score of the QuantumNaiveBayes classifier is 0.64
The recall score of the QuantumNaiveBayes classifier is 0.41
The specificity score of the QuantumNaiveBayes classifier is 0.82
The npv score of the QuantumNaiveBayes classifier is 0.64
The information level is: 0.64


In [ ]:
# Visualize circuit structure
def show_circuit_example():
    """Display an example quantum circuit"""
    example_modifiers = [1.4, 1.3]
    qr = QuantumRegister(7)
    cr = ClassicalRegister(1)
    qc_example = QuantumCircuit(qr, cr)

    # Show the basic structure
    qc_example.ry(prob_to_angle(prob_survival), 6)
    qc_example.x(6)
    qc_example.cry(prob_to_angle(prob_survival / (1 - prob_survival)), 6, 5)
    qc_example.measure(6, 0)

    print("\nExample quantum circuit structure:")
    print(qc_example.draw())
    return qc_example

In [ ]:
show_circuit_example()


Example quantum circuit structure:
                                           
q52_0: ────────────────────────────────────
                                           
q52_1: ────────────────────────────────────
                                           
q52_2: ────────────────────────────────────
                                           
q52_3: ────────────────────────────────────
                                           
q52_4: ────────────────────────────────────
                          ┌────────────┐   
q52_5: ───────────────────┤ Ry(1.8192) ├───
       ┌────────────┐┌───┐└─────┬──────┘┌─┐
q52_6: ┤ Ry(1.3363) ├┤ X ├──────■───────┤M├
       └────────────┘└───┘              └╥┘
c51: 1/══════════════════════════════════╩═
                                         0 
